In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)


from time import sleep
import requests
import geopandas as gpd

In [2]:
from pathlib import Path

current_dir = Path().resolve()
parent_dir = current_dir.parent
print(parent_dir)

/Users/keenansmacbookairm3/Documents/GitHub/nyc_2024


In [ ]:
raw_data_folder= parent_dir/"raw_data/"

In [11]:
# Import 2020 election results that have already been saved in a csv file
df = pd.read_csv(raw_data_folder / "2020_official.csv", index_col=False, header=None)

# Keep only relevant columns and rename them
df = df[[11, 12, 13, 20, 21]]
df.columns=['assembly_district', 'election_district', 'county', 'candidate', 'vote_count']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79755 entries, 0 to 79754
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   assembly_district  79755 non-null  int64 
 1   election_district  79755 non-null  int64 
 2   county             79755 non-null  object
 3   candidate          79755 non-null  object
 4   vote_count         79755 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.0+ MB


/var/folders/f2/xbsrplkj23z71l_fw6890_hr0000gn/T/ipykernel_99025/2457991774.py:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(raw_data_folder / "2020_official.csv", index_col=False, header=None)


In [12]:
df[['assembly_district', 'election_district']] = df[['assembly_district', 'election_district']].astype(str)

df['vote_count'] = df['vote_count'].astype(str)
df['vote_count'] = df['vote_count'].str.replace(',', '', regex=False).str.strip()
df['vote_count'] = pd.to_numeric(df['vote_count'], errors='coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79755 entries, 0 to 79754
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   assembly_district  79755 non-null  object
 1   election_district  79755 non-null  object
 2   county             79755 non-null  object
 3   candidate          79755 non-null  object
 4   vote_count         79755 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


In [13]:
# A function to clean up the election district number 
def format_district(district_code):
    # Check the length and add zeros accordingly
    if len(district_code) == 1:
        return '00' + district_code  # Add two zeros for single digits
    elif len(district_code)== 2:
        return '0' + district_code   # Add one zero for two-digit numbers
    else:
        return district_code 

df['ed_code'] = df.assembly_district+df.election_district.apply(format_district)
df.head()

,assembly_district,election_district,county,candidate,vote_count,ed_code
0,65,1,New York,Public Counter,393,65001
1,65,1,New York,Manually Counted Emergency,0,65001
2,65,1,New York,Absentee / Military,263,65001
3,65,1,New York,Federal,7,65001
4,65,1,New York,Special Presidential,0,65001


In [14]:
# Create a pivot table that summarizes vote count by election district and candidate
pivot_table = df.pivot(index='ed_code', columns='candidate', values='vote_count')

# Set election district as the index for the pivot table
pivot_table = pivot_table.reset_index().set_index('ed_code')

pivot_table.head()

candidate,Absentee / Military,Affidavit,Brock Pierce / Karla Ballard (Independence),Donald J. Trump / Michael R. Pence (Conservative),Donald J. Trump / Michael R. Pence (Republican),Federal,Howie Hawkins / Angela Nicole Walker (Green),Jo Jorgensen / Jeremy Cohen (Libertarian),Joseph R. Biden / Kamala D. Harris (Democratic),Joseph R. Biden / Kamala D. Harris (Working Families),Manually Counted Emergency,Public Counter,Scattered,Special Presidential
ed_code,,,,,,,,,,,,,,
23001,121.0,3.0,0.0,141.0,510.0,0.0,5.0,10.0,216.0,27.0,0.0,797.0,6.0,0.0
23002,124.0,5.0,0.0,104.0,590.0,1.0,1.0,9.0,219.0,24.0,0.0,821.0,1.0,0.0
23003,96.0,5.0,2.0,80.0,556.0,4.0,8.0,10.0,199.0,16.0,0.0,772.0,2.0,0.0
23004,16.0,3.0,0.0,11.0,100.0,0.0,0.0,0.0,48.0,1.0,0.0,141.0,NaN,0.0
23005,187.0,18.0,0.0,41.0,445.0,3.0,5.0,8.0,286.0,23.0,0.0,606.0,2.0,0.0


In [15]:
pivot_table['precinct_total'] = pivot_table.iloc[:, list(range(2, 5)) + list(range(6, 10))].sum(axis=1)

pivot_table['trump_total'] = pivot_table.iloc[:,3:5].sum(axis=1)
pivot_table['trump_share'] = pivot_table.trump_total/ pivot_table.precinct_total

pivot_table['biden_total'] = pivot_table.iloc[:,8:10].sum(axis=1)
pivot_table['biden_share'] = pivot_table.biden_total/ pivot_table.precinct_total


pivot_table = pivot_table.iloc[:, -5:].reset_index()
pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5901 entries, 0 to 5900
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ed_code         5901 non-null   object 
 1   precinct_total  5901 non-null   float64
 2   trump_total     5901 non-null   float64
 3   trump_share     5631 non-null   float64
 4   biden_total     5901 non-null   float64
 5   biden_share     5631 non-null   float64
dtypes: float64(5), object(1)
memory usage: 276.7+ KB


In [19]:
geo_file = 'NYS_Elections_Districts_and_Polling_Locations_-3344009424923235749.geojson'

gdf = gpd.read_file(raw_data_folder / geo_file)

gdf.columns = gdf.columns.str.lower()
gdf = gdf[gdf.municipality.str.contains('New York City')]

gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 4345 entries, 8955 to 13299
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   objectid           4345 non-null   int32   
 1   county             4345 non-null   object  
 2   municipality       4345 non-null   object  
 3   election_district  4345 non-null   object  
 4   geometry           4345 non-null   geometry
dtypes: geometry(1), int32(1), object(3)
memory usage: 186.7+ KB


In [20]:
new_gdf = pivot_table.merge(gdf[['county', 'geometry', 'election_district']], 
                      left_on='ed_code', 
                      right_on= 'election_district', 
                      how='right',
                      suffixes=('_result', '_map')
                      ).drop(columns='ed_code')


In [26]:
# Save only precincts that both candiates receivced voets
new_gdf = new_gdf[(new_gdf.biden_total > 0) & (new_gdf.trump_total > 0)]
new_gdf = gpd.GeoDataFrame(new_gdf)

In [29]:
processed_data_folder = parent_dir/ 'semi_processed_data'

new_gdf.to_file(processed_data_folder /'2020_official_geojson.geojson', driver="GEOJSON")